In [33]:
import pandas as pd
pd.options.display.max_rows = 10000
pd.set_option('display.max_colwidth', -1)
from collections import Counter

In [19]:
graph='2006.csv'
network=graph

In [20]:
df=pd.read_csv(network)

In [21]:
dg=df[['id','modularity_class']].groupby('modularity_class').count()
dg=dg.reset_index(drop=True)
dg.rename(columns={'id':'cluster size'},inplace=True)

In [22]:
df['mod_size']=df.modularity_class.apply(lambda x: dg.iloc[x])

In [23]:
#df.sort_values(['mod_size','degree'],ascending=False)[['label','modularity_class']]

In [24]:
docs=df.label

In [25]:
import re
from nltk import stem
porter = stem.porter.PorterStemmer()

docs_cleaned = [[re.sub('[.!,;?()\W\d]+', '', word) for word in doc.lower().split()] for doc in docs]

In [26]:
stop= open ('stopList.txt').read()
stop_list = set(stop.replace("\n", " ").split())
docs_stemed = [[porter.stem(word) for word in document if word not in stop_list] for document in docs_cleaned]

In [30]:
df['ind']=df.index

In [31]:
df['terms']=df.ind.apply(lambda x: docs_stemed[x])

In [34]:
dg['keyword']=''
dg['key terms']=''

for mod in dg.index:
    mod_terms=[]
    for terms in df[df.modularity_class==mod].terms:
        for term in terms:
            mod_terms.append(term)
            
    words = [word for word in mod_terms if word != '']                
    words_counter = Counter(words)
    top_words=[word for word in words_counter.most_common(5) if word[1]>1]
    dg.set_value(mod,'keyword',top_words)
    
    terms = [words[i]+' '+words[i+1] for i in range(len(words)-1)]
    terms_counter= Counter(terms)
    top_terms=[term for term in terms_counter.most_common(5) if term[1]>1]
    dg.set_value(mod,'key terms',top_terms)

In [35]:
density=df.groupby(['modularity_class'])['triangles'].mean()
dg['cluster density']=(density/max(density)).round(2)
dg=dg.sort_values('cluster density',ascending=False).reset_index(drop=True)

In [36]:
#dg.to_csv('test.csv', encoding='ascii')
dg

,cluster size,keyword,key terms,cluster density
0,49,"[(knowledg, 41), (manag, 18), (organiz, 11), (process, 8), (system, 6)]","[(knowledg manag, 14), (organiz knowledg, 3), (manag system, 3), (electron economi, 2), (knowledg repositori, 2)]",1.00
1,55,"[(group, 31), (support, 20), (system, 14), (electron, 9), (decis, 8)]","[(support system, 14), (group support, 13), (electron brainstorm, 7), (decis make, 5), (make group, 2)]",0.71
2,46,"[(measur, 20), (user, 17), (system, 13), (satisfact, 11), (servic, 10)]","[(user satisfact, 7), (servic qualiti, 7), (measur system, 5), (system servic, 5), (user particip, 3)]",0.37
3,32,"[(project, 18), (softwar, 10), (system, 8), (manag, 7), (technolog, 7)]","[(softwar project, 6), (sourc softwar, 3), (project manag, 3), (softwar develop, 3), (system develop, 3)]",0.26
4,21,"[(job, 7), (career, 7), (profession, 6), (skill, 4), (turnov, 4)]","[(system profession, 3), (career orient, 2), (technolog worker, 2), (job skill, 2), (gender differ, 2)]",0.23
5,39,"[(technolog, 23), (adopt, 14), (accept, 13), (model, 13), (usag, 8)]","[(technolog accept, 8), (belief technolog, 3), (accept model, 3), (user accept, 3), (technolog usag, 3)]",0.21
6,31,"[(databas, 12), (design, 6), (requir, 5), (analysi, 5), (conceptu, 5)]","[(databas design, 3), (requir elicit, 3), (conceptu model, 2), (data represent, 2), (empir test, 2)]",0.21
7,39,"[(comput, 15), (learn, 11), (train, 10), (collabor, 7), (environ, 6)]","[(comput selfefficaci, 4), (collabor learn, 2), (learn comput, 2), (longitudin field, 2), (train method, 2)]",0.21
8,22,"[(web, 11), (site, 6), (onlin, 6), (consum, 5), (design, 4)]","[(web site, 5), (site usabl, 2), (design web, 2), (onlin consum, 2), (consum percept, 2)]",0.20
9,31,"[(standard, 13), (technolog, 8), (edi, 7), (electron, 6), (system, 5)]","[(electron data, 3), (data interchang, 3), (open system, 2), (standard make, 2), (switch cost, 2)]",0.17
